# One-Class Classification with Tweet Embeddings

In [1]:
# import modules
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.base import TransformerMixin
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import re
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer

## Load data

In [2]:
xls = pd.ExcelFile('../../../ForLastExpt.xlsx')

In [3]:
class1_train = pd.read_excel(xls, 'Class=1 Training')
class1_tune = pd.read_excel(xls, 'Class=1 Validation')
class1_test_pt1 = pd.read_excel(xls, 'Class=1 Test Part-1')
class1_test_pt2 = pd.read_excel(xls, 'Class=1 Test Part-2')
class0_train = pd.read_excel(xls, 'Class=0 Training')
class0_tune = pd.read_excel(xls, 'Class=0 Validation')
class0_test_pt1 = pd.read_excel(xls, 'Class=0 Test Part-1')
class0_test_pt2 = pd.read_excel(xls, 'Class=0 Test Part-2')

In [4]:
# create outlier target values

# class1_train
outlier = []
for i in class1_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_train['outlier_target'] = outlier

# class1_tune
outlier = []
for i in class1_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_tune['outlier_target'] = outlier

# class1_test_pt1
outlier = []
for i in class1_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt1['outlier_target'] = outlier

# class1_test_pt2
outlier = []
for i in class1_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt2['outlier_target'] = outlier

# class0_train
outlier = []
for i in class0_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_train['outlier_target'] = outlier

# class0_tune
outlier = []
for i in class0_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_tune['outlier_target'] = outlier

# class0_test_pt1
outlier = []
for i in class0_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt1['outlier_target'] = outlier

# class0_test_pt2
outlier = []
for i in class0_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt2['outlier_target'] = outlier

In [5]:
train_fold1 = class0_train
tune_fold1 = pd.concat([class1_tune, class0_tune, class1_train], ignore_index = True)
test_fold1 = pd.concat([class1_test_pt1, class1_test_pt2, class0_test_pt1, class0_test_pt2], ignore_index = True)

train_fold2 = class0_test_pt1
tune_fold2 = pd.concat([class1_test_pt2, class1_test_pt1, class0_test_pt2], ignore_index = True)
test_fold2 = pd.concat([class1_tune, class0_tune, class1_train, class0_train], ignore_index = True)

In [6]:
class1_train

,Is_Unreliable,Category,Tweet,outlier_target
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...,-1
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government...",-1
2,1,"1, 4, 9",It was only a matter of time before the mainst...,-1
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...,-1
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...,-1
...,...,...,...,...
95,1,"11, 10",EOIR has ordered immigration court staff to re...,-1
96,1,10,President - who said last night that #coronavi...,-1
97,1,11,"Mashhad, NE #Iran Man collapses possibly due t...",-1
98,1,"10, 11",Pence caught in lies about insurance covering ...,-1


## Derive text vectors from word embeddings

In [7]:
embeddings = pd.read_csv('word_embed_500.csv', index_col = 0)
embeddings

,Comp 1,Comp 2,Comp 3,Comp 4,Comp 5,Comp 6,Comp 7,Comp 8,Comp 9,Comp 10,...,Comp 491,Comp 492,Comp 493,Comp 494,Comp 495,Comp 496,Comp 497,Comp 498,Comp 499,Comp 500
!,-0.002445,0.009159,0.004795,-0.018603,0.005400,-0.003888,-0.008644,-0.013260,-0.023465,0.003785,...,0.006786,-0.097490,0.026916,0.002518,0.005202,-0.015148,-0.012731,0.000193,-0.023189,-0.015074
#,0.001323,-0.013805,0.019507,-0.003647,-0.015684,-0.002614,-0.003105,0.022333,0.040944,0.008360,...,-0.084790,0.008607,-0.002623,0.003791,-0.005258,0.048565,0.010139,-0.083814,0.023770,0.015932
(,-0.000766,-0.002131,0.001162,-0.001840,0.000518,-0.002776,0.000748,-0.006843,-0.001281,0.005484,...,0.003895,-0.000447,0.001618,0.006980,-0.000497,0.003171,-0.005241,0.002066,0.013100,0.002086
),-0.000342,-0.001425,0.002255,-0.000580,-0.000688,-0.002130,-0.000360,-0.004394,-0.000817,0.004866,...,-0.001170,0.003189,0.000424,0.006401,-0.000815,-0.008037,-0.002319,0.002666,-0.007344,0.006109
",",0.000345,-0.031475,-0.005695,-0.002895,0.000669,-0.005995,0.000626,0.005936,-0.022008,0.025221,...,0.070594,-0.007403,-0.025496,-0.001728,-0.009012,0.013035,0.057475,-0.004225,0.022683,0.032053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‘,-0.005880,-0.004665,-0.214563,-0.011658,-0.005440,-0.004683,-0.007761,0.003650,0.050089,0.002729,...,0.005793,0.004386,-0.046885,0.002524,0.005406,-0.017934,0.006290,0.000401,0.006088,0.001553
’,-0.002839,-0.025199,-0.017961,0.005492,0.001238,0.011790,-0.002090,-0.010750,0.109708,-0.005302,...,-0.076606,-0.035435,0.019013,0.017229,0.029816,0.049977,-0.025344,0.003653,-0.001604,0.176081
“,0.000341,0.011505,0.003999,-0.005339,-0.003270,0.003572,0.001421,-0.008352,0.001352,0.003367,...,0.003169,0.011799,-0.004631,-0.002775,-0.002354,-0.005303,-0.004375,0.010388,-0.010847,-0.007249
”,-0.000290,0.021694,0.006578,-0.004365,0.000902,-0.009067,-0.001975,-0.004271,0.002729,-0.001118,...,-0.001370,-0.004085,-0.009635,-0.002395,-0.001476,-0.007520,0.000675,-0.003917,0.006633,-0.018571


In [8]:
# to convert contractions picked up by word_tokenize() into full words
contractions = {
    "n't": 'not',
    "'ve": 'have',
    "'s": 'is', # note that this will include possessive nouns
    'gonna': 'going to',
    'gotta': 'got to',
    "'d": 'would',
    "'ll": 'will',
    "'re": 'are',
    "'m": 'am',
    'wanna': 'want to'
}

# to convert nltk_pos tags to wordnet-compatible PoS tags
def convert_pos_wordnet(tag):
    tag_abbr = tag[0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
                
    if tag_abbr in tag_dict:
        return tag_dict[tag_abbr]

In [9]:
def get_text_vectors(word_embeddings, # pandas dataframe with row index = vocabulary
                     text_list, # list of strings
                     remove_stopwords = True,
                     lowercase = True,
                     lemmatize = True,
                     add_start_end_tokens = True):
    
    lemmatizer = WordNetLemmatizer()
    
    for k in range(len(text_list)):
        text = text_list[k]
        text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
        text_vec = np.zeros(word_embeddings.shape[1])
        words = word_tokenize(text)
        tracker = 0 # to track whether we've encountered a word for which we have an embedding (in each tweet)
        
        if remove_stopwords:
            clean_words = []
            for word in words:
                if word.lower() not in set(stopwords.words('english')):
                    clean_words.append(word)
            words = clean_words

        if lowercase:
            clean_words = []
            for word in words:
                clean_words.append(word.lower())

            words = clean_words

        if lemmatize:
            clean_words = []
            for word in words:
                PoS_tag = pos_tag([word])[0][1]

                # to change contractions to full word form
                if word in contractions:
                    word = contractions[word]

                if PoS_tag[0].upper() in 'JNVR':
                    word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                else:
                    word = lemmatizer.lemmatize(word)

                clean_words.append(word)

            words = clean_words

        if add_start_end_tokens:
            words = ['<START>'] + words + ['<END>']
        
        for i in range(len(words)):
            word = words[i]
            if word in word_embeddings.index.tolist():
                word_embed_vec = word_embeddings.loc[word].values
                if tracker == 0:
                    text_matrix = word_embed_vec
                else:
                    text_matrix = np.vstack((text_matrix, word_embed_vec))
                    
                # only increment if we have come across a word in the embeddings dictionary
                tracker += 1
                    
        for j in range(len(text_vec)):
            text_vec[j] = text_matrix[:,j].mean()
            
        if k == 0:
            full_matrix = text_vec
        else:
            full_matrix = np.vstack((full_matrix, text_vec))
            
    return full_matrix

In [10]:
train_fold1_X = get_text_vectors(embeddings, train_fold1['Tweet'])
train_fold1_y = train_fold1['outlier_target']

tune_fold1_X = get_text_vectors(embeddings, tune_fold1['Tweet'])
tune_fold1_y = tune_fold1['outlier_target']

test_fold1_X = get_text_vectors(embeddings, test_fold1['Tweet'])
test_fold1_y = test_fold1['outlier_target']

train_fold2_X = get_text_vectors(embeddings, train_fold2['Tweet'])
train_fold2_y = train_fold2['outlier_target']

tune_fold2_X = get_text_vectors(embeddings, tune_fold2['Tweet'])
tune_fold2_y = tune_fold2['outlier_target']

test_fold2_X = get_text_vectors(embeddings, test_fold2['Tweet'])
test_fold2_y = test_fold2['outlier_target']

## Binary classification: two-fold CV

In [11]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

In [12]:
# SVC hyperparams to optimize
kernel = ['rbf', 'linear', 'poly', 'sigmoid']

### Fold 1

In [13]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold1_X, train_fold1_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold1_X)
    f1 = f1_score(tune_fold1_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='linear')

In [14]:
tune_f1

[0.6633465269018312, 0.674118984463812, 0.6296296296296295, 0.6641651809776524]

In [15]:
# testing
test_scores = dict()
test_predict = opt_model.predict(test_fold1_X)
test_predict_proba = opt_model.decision_function(test_fold1_X)
test_scores['auc'] = [roc_auc_score(test_fold1_y, test_predict_proba, average = 'macro')]
test_scores['accuracy'] = [accuracy_score(test_fold1_y, test_predict)]
test_scores['recall'] = [recall_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['precision'] = [precision_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['f1'] = [f1_score(test_fold1_y, test_predict, average = 'macro')]
test_scores

{'auc': [0.6858163265306122],
 'accuracy': [0.6285714285714286],
 'recall': [0.6285714285714286],
 'precision': [0.6456984273820536],
 'f1': [0.6173254835996635]}

### Fold 2

In [16]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold2_X, train_fold2_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold2_X)
    f1 = f1_score(tune_fold2_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='sigmoid')

In [17]:
tune_f1

[0.47514665020067925, 0.52, 0.48499743232338055, 0.5286195286195287]

In [18]:
# testing
test_predict = opt_model.predict(test_fold2_X)
test_predict_proba = opt_model.decision_function(test_fold2_X)
test_scores['auc'].append(roc_auc_score(test_fold2_y, test_predict_proba, average = 'macro'))
test_scores['accuracy'].append(accuracy_score(test_fold2_y, test_predict))
test_scores['recall'].append(recall_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['precision'].append(precision_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['f1'].append(f1_score(test_fold2_y, test_predict))
test_scores

{'auc': [0.6858163265306122, 0.6893367346938775],
 'accuracy': [0.6285714285714286, 0.6321428571428571],
 'recall': [0.6285714285714286, 0.6321428571428571],
 'precision': [0.6456984273820536, 0.7217655621200445],
 'f1': [0.6173254835996635, 0.46073298429319376]}

### Average results

In [19]:
sum(test_scores['auc'])/2

0.6875765306122448

In [20]:
sum(test_scores['accuracy'])/2

0.6303571428571428

In [21]:
sum(test_scores['recall'])/2

0.6303571428571428

In [22]:
sum(test_scores['precision'])/2

0.6837319947510491

In [23]:
sum(test_scores['f1'])/2

0.5390292339464287